<a href="https://colab.research.google.com/github/maya-halevy/Kaggle-Contrails/blob/main/trails_eda_M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from scipy.stats import pearsonr
import pandas as pd
from scipy.stats import spearmanr


In [5]:
# SET PATHS!!!!

PATH_train_fifth_sample = '/content/drive/MyDrive/contrails_data/train_xs/1026930305418425156/band_12.npy'
PATH_mask_human_fifth_sample = '/content/drive/MyDrive/contrails_data/train_xs/1026930305418425156/human_individual_masks.npy'
PATH_mask_auto_fifth_sample = 'content/drive/MyDrive/contrails_data/train_xs/1026930305418425156/human_pixel_masks.npy'
one_sample_folder_path = '/content/drive/MyDrive/contrails_data/train_xs/1026930305418425156'

# Set the path to the train folder
train_folder = '/content/drive/MyDrive/contrails_data/train_xs'



## check size of the dataset

In [6]:
def count_folders(subfolder_path):
    """
    Counts the number of folders within a subfolder and prints the count.

    Args:
        subfolder_path (str): Path to the subfolder.
    """
    if not os.path.isdir(subfolder_path):
        raise ValueError("Invalid subfolder path.")

    count = 0
    for item_name in os.listdir(subfolder_path):
        item_path = os.path.join(subfolder_path, item_name)
        if os.path.isdir(item_path):
            count += 1

    print("Number of folders:", count)

In [7]:
count_folders(train_folder)

Number of folders: 514


## check for duplicates

In [8]:
def check_subfolder_names(folder_path):
    """
    Checks whether all subfolder names in a folder are unique and prints the duplicate names if any.

    Args:
        folder_path (str): Path to the folder.
    """
    if not os.path.isdir(folder_path):
        raise ValueError("Invalid folder path.")

    subfolder_names = []
    duplicate_names = []

    for subfolder_name in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, subfolder_name)

        if os.path.isdir(subfolder_path):
            if subfolder_name in subfolder_names:
                duplicate_names.append(subfolder_name)
            else:
                subfolder_names.append(subfolder_name)

    if duplicate_names:
        print("Duplicate subfolder names:")
        for duplicate_name in duplicate_names:
            print(duplicate_name)
    else:
        print("No duplicate subfolder names found.")



In [9]:
check_subfolder_names(train_folder)

No duplicate subfolder names found.


## print a single random band file

In [10]:
train_image = np.load(PATH_train_fifth_sample)

# Access the data
display(train_image[:, :, 4])


array([[246.7658 , 245.80508, 248.91423, ..., 253.13512, 253.07115,
        253.56847],
       [247.18756, 248.1787 , 251.60426, ..., 253.24692, 254.02245,
        254.77922],
       [248.7403 , 250.6605 , 254.115  , ..., 253.88829, 254.40623,
        255.10202],
       ...,
       [265.743  , 265.61783, 265.67334, ..., 264.09637, 264.06427,
        264.0653 ],
       [266.02524, 265.94333, 265.68652, ..., 264.0709 , 264.06427,
        264.0675 ],
       [266.0399 , 266.00916, 265.67612, ..., 264.04077, 264.09824,
        264.08847]], dtype=float32)

## display the shape of the single random band file

In [11]:
train_image.shape

(256, 256, 8)

## display the shape of the corresponding human mask file

In [12]:
sample_mask_human = np.load(PATH_mask_human_fifth_sample)


sample_mask_human.shape


(256, 256, 1, 4)

## display the shape of the corresponding auto mask file

In [13]:
sample_mask_auto = np.load(PATH_mask_auto_fifth_sample)


sample_mask_auto.shape


FileNotFoundError: ignored

In [ ]:
def visualize_band_images(folder_path, figsize=(10, 10), cmap='Spectral'):
    # Set the figure size
    plt.figure(figsize=figsize)

    # Loop over the bands
    for i in range(9):
        # Load the band data
        band_data = np.load(f'{folder_path}/band_{i+8:02d}.npy')

        # Extract the 5th dimension for visualization
        band_data_5th = band_data[:, :, 4]

        # Create a subplot for each band
        plt.subplot(3, 3, i+1)

        # Use seaborn to visualize the image
        sns.heatmap(band_data_5th, cmap=cmap, cbar=False)

        # Set the title of the subplot
        plt.title(f'Band {i+8}')

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()


## visualize all bands in one single folder

In [ ]:
#for every 5h image in a band
visualize_band_images(one_sample_folder_path)


In [ ]:
def visualize_individual_masks(folder_path):
    # Load the individual masks
    masks_data = np.load(f'{folder_path}/human_individual_masks.npy')

    # Get the number of individual labellers
    num_labellers = masks_data.shape[3]

    # Compute the grid size for the subplots
    grid_size = int(np.ceil(np.sqrt(num_labellers)))

    # Create a grid of subplots
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))

    # Flatten the axes for easy iteration
    axes = axes.flatten()

    # Loop over the labellers
    for i in range(num_labellers):
        # Extract the mask for the labeller
        mask = masks_data[:, :, 0, i]

        # Use seaborn to visualize the mask on the appropriate subplot
        sns.heatmap(mask, cmap='Blues', cbar=False, linewidths=0.0, ax=axes[i])

        # Set the title of the subplot
        axes[i].set_title(f'Labeller {i+1}')

    # Remove any unused subplots
    for i in range(num_labellers, len(axes)):
        fig.delaxes(axes[i])

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()


## visualize corresponding human masks

In [ ]:
visualize_individual_masks(one_sample_folder_path)

In [ ]:
def visualize_pixel_mask(folder_path):
    # Load the pixel masks
    masks_data = np.load(f'{folder_path}/human_pixel_masks.npy')

    # Extract the mask
    mask = masks_data[:, :, 0]

    # Use seaborn to visualize the mask
    sns.heatmap(mask, cmap='Blues', cbar=False, linewidths=0.0)
    plt.title('Pixel Masks')

    # Display the plot
    plt.show()


## visualize corresponding auto mask

In [ ]:
visualize_pixel_mask(one_sample_folder_path)

In [ ]:
def visualize_band_images(folder_path, figsize=(20, 20), dpi=300):
    # Import the required libraries
    import numpy as np
    import matplotlib.pyplot as plt

    # Determine the number of bands and time points from the first band file
    first_band_data = np.load(f'{folder_path}/band_08.npy')
    num_time_points = first_band_data.shape[2]

    # Set the figure size and dpi
    plt.figure(figsize=figsize, dpi=dpi)

    # Loop over the bands
    for i in range(9):
        # Load the band data
        band_data = np.load(f'{folder_path}/band_{i+8:02d}.npy')

        # Loop over the time points
        for j in range(num_time_points):
            # Extract the image for the current time point
            band_image = band_data[:, :, j]

            # Create a subplot for this band and time point
            plt.subplot(9, num_time_points, i*num_time_points + j + 1)

            # Display the image
            plt.imshow(band_image, cmap='Spectral')
            plt.axis('off')  # Hide the axis

            # Set the title of the subplot
            if i == 0:
                plt.title(f'Time {j+1}')
            if j == 0:
                plt.ylabel(f'Band {i+8}')

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()


## visuzlize all the bands in all the time points in a single folder

In [ ]:
visualize_band_images(one_sample_folder_path)

In [ ]:

def check_value_ranges(folder_path):
    # Iterate over the bands
    for band_number in range(8, 17):
        # Load the band data
        band_data = np.load(f'{folder_path}/band_{band_number:02d}.npy')

        # Calculate the minimum and maximum values
        min_value = np.min(band_data[:, :, 4])
        max_value = np.max(band_data[:, :, 4])

        print(f'Band {band_number}: Min Value: {min_value}, Max Value: {max_value}')


## look at the value ranges for each band in a single folder

In [ ]:
#for every 5h image in a band
check_value_ranges(one_sample_folder_path)

## check the distribution of masks (yes no contrails) for the entire dataset

In [ ]:


# Get the paths of all the human_pixel_masks.npy files
mask_paths = glob.glob(f'{train_folder}/**/human_pixel_masks.npy', recursive=True)

# Count the number of files with all zeros
num_all_zeros = 0

# Iterate over the mask files
for mask_path in mask_paths:
    # Load the mask data using memmap
    mask_data = np.load(mask_path, mmap_mode='r')

    # Check if all elements in the mask are zero
    if np.all(mask_data == 0):
        num_all_zeros += 1

# Calculate the percentage of files with all zeros
percentage_all_zeros = (num_all_zeros / len(mask_paths)) * 100

# Print the results
print("Number of auto masks with all zeros:", num_all_zeros)
print("Percentage of auto masks with all zeros:", percentage_all_zeros)


In [ ]:
def plot_band_distribution(folder_path):
    for band_number in range(8, 17):
        # Load the band data
        band_data = np.load(f'{folder_path}/band_{band_number:02d}.npy')

        # Flatten the band data
        flattened_data = band_data[:, :, 4].flatten()

        # Create the histogram trace
        histogram_trace = go.Histogram(x=flattened_data, nbinsx=50)

        # Create the layout
        layout = go.Layout(
            title=f'Distribution of Band {band_number} Data',
            xaxis=dict(title='Pixel Value'),
            yaxis=dict(title='Frequency'),
            bargap=0.1,
            bargroupgap=0.1
        )

        # Create the figure
        fig = go.Figure(data=[histogram_trace], layout=layout)

        # Show the plot
        fig.show()


## plot distribution of values for every bans in a single folder

In [ ]:
#for every 5h image in a band
plot_band_distribution(one_sample_folder_path)

## plot min vs max values distribution per band for the entire dataset

In [ ]:

# Initialize empty lists to store min and max values
min_values, max_values = [], []

# Get the list of subfolders in the train folder
subfolders = [f for f in os.listdir(train_folder) if not f.startswith('.') and os.path.isdir(os.path.join(train_folder, f))]

# Iterate over the bands
for band_number in range(8, 17):
    # Initialize empty lists for min and max values of the current band
    band_min_values, band_max_values = [], []

    # Specify the paths to the .npy files for the band
    file_paths = [os.path.join(train_folder, subdir, f'band_{band_number:02d}.npy') for subdir in subfolders]

    # Iterate over the .npy files and calculate the min and max
    for file_path in file_paths:
        band_data = np.load(file_path)[:, :, 4]  # Extract the 5th dimension directly
        band_min_values.append(np.min(band_data))
        band_max_values.append(np.max(band_data))

    # Append the min and max values of the current band to the main arrays
    min_values.append(band_min_values)
    max_values.append(band_max_values)

    # Create the plotly figure for min and max values of the current band
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=band_min_values, name='Min Values', nbinsx=50))
    fig.add_trace(go.Histogram(x=band_max_values, name='Max Values', nbinsx=50))
    fig.update_layout(
        title=f'Distribution of Min and Max Values for Band {band_number}',
        xaxis_title='Value',
        yaxis_title='Count',
        bargap=0.1,
        bargroupgap=0.1
    )
    fig.show()

# Convert the min and max values lists to numpy arrays
min_values = np.array(min_values)
max_values = np.array(max_values)


In [ ]:
# import os
# import random
# import shutil

# # Set the paths
# train_folder = '/Volumes/Velik/contrails/train'
# new_dataset_folder = '/Volumes/Velik/contrails/train_xs'

# # Create the new dataset folder
# os.makedirs(new_dataset_folder, exist_ok=True)

# # List all the subfolders in the train folder
# subfolders = os.listdir(train_folder)

# # Calculate the number of subfolders to select (10% of the total)
# num_subfolders = int(len(subfolders) * 0.025)

# # Shuffle the subfolders randomly
# random.shuffle(subfolders)

# # Select the first num_subfolders subfolders
# selected_subfolders = subfolders[:num_subfolders]

# # Count the number of subfolders with all-zero human_pixel_masks.npy
# num_all_zeros_subfolders = 0

# # Iterate over the selected subfolders
# for subfolder in selected_subfolders:
#     # Create the path for the source subfolder
#     source_path = os.path.join(train_folder, subfolder)

#     # Copy the source subfolder to the new dataset folder
#     shutil.copytree(source_path, os.path.join(new_dataset_folder, subfolder))

#     # Check if the subfolder has all-zero human_pixel_masks.npy
#     mask_file_path = os.path.join(source_path, 'human_pixel_masks.npy')
#     mask_data = np.load(mask_file_path)

#     if np.all(mask_data == 0):
#         num_all_zeros_subfolders += 1

# # Calculate the percentage of subfolders with all-zero human_pixel_masks.npy
# percentage_all_zeros = (num_all_zeros_subfolders / num_subfolders) * 100

# # Print the results
# print(f"Number of selected subfolders: {num_subfolders}")
# print(f"Number of subfolders with all-zero human_pixel_masks.npy: {num_all_zeros_subfolders}")
# print(f"Percentage of subfolders with all-zero human_pixel_masks.npy: {percentage_all_zeros:.1f}%")


## plot distribution оf mean and std of each band of the entire dataset

In [ ]:
# Initialize empty arrays to store mean and standard deviation values
mean_values = []
std_values = []

# Iterate over the bands
for band_number in range(8, 17):
    # Initialize empty arrays for mean and standard deviation values of the current band
    band_mean_values = []
    band_std_values = []

    # Iterate over the subfolders in the train folder
    for subdir in os.listdir(train_folder):
        subdir_path = os.path.join(train_folder, subdir)

        if not os.path.isdir(subdir_path) or subdir.startswith('.'):
            continue

        # Specify the path to the .npy file for the band
        file_path = os.path.join(subdir_path, f'band_{band_number:02d}.npy')

        # Load the .npy file
        band_data = np.load(file_path)

        # Extract the 5th dimension for mean and standard deviation calculations
        band_data_5th = band_data[:, :, 4]

        # Compute the mean and standard deviation values for the current band
        band_mean = np.mean(band_data_5th)
        band_std = np.std(band_data_5th)

        # Append the mean and standard deviation values to the respective arrays
        band_mean_values.append(band_mean)
        band_std_values.append(band_std)

    # Append the mean and standard deviation values of the current band to the main arrays
    mean_values.append(band_mean_values)
    std_values.append(band_std_values)

    # Create the plotly figure for the means of the current band
    fig_mean = go.Figure()
    fig_mean.add_trace(go.Histogram(x=band_mean_values, name='Mean Values', nbinsx=50, marker_color='lightblue'))
    fig_mean.update_layout(title=f'Distribution of Mean Values for Band {band_number}', bargap=0.1)
    fig_mean.show()

    # Create the plotly figure for the standard deviations of the current band
    fig_std = go.Figure()
    fig_std.add_trace(go.Histogram(x=band_std_values, name='Standard Deviation Values', nbinsx=50, marker_color='lightcoral'))
    fig_std.update_layout(title=f'Distribution of Standard Deviation Values for Band {band_number}', bargap=0.1)
    fig_std.show()

# Convert the mean and standard deviation values arrays to numpy arrays
mean_values = np.array(mean_values)
std_values = np.array(std_values)


## plot Pearson and Spearman correlation between bands

In [ ]:
# Define the band files
band_files = ['band_08.npy', 'band_09.npy', 'band_10.npy', 'band_11.npy', 'band_12.npy', 'band_13.npy', 'band_14.npy', 'band_15.npy', 'band_16.npy']

# Initialize a dictionary to store the flattened pixel values for each band
band_data = {band_file: [] for band_file in band_files}

# Iterate over each sample in the dataset
for sample_folder in tqdm(os.listdir(train_folder)):
    # Ensure the folder is a directory
    if os.path.isdir(os.path.join(train_folder, sample_folder)):
        # Iterate over each band
        for band_file in tqdm(band_files, leave=False, desc=f'Processing {sample_folder}'):
            # Load the band data
            band_array = np.load(os.path.join(train_folder, sample_folder, band_file))
            # Flatten the 5th image and append it to the list for the current band
            band_data[band_file].extend(band_array[:, :, 4].flatten())

# Convert the lists to numpy arrays
for band_file in tqdm(band_files, desc='Converting to numpy arrays'):
    band_data[band_file] = np.array(band_data[band_file])

# Create a DataFrame to store the correlation values
correlation_df_pearson = pd.DataFrame(index=band_files, columns=band_files)
correlation_df_spearman = pd.DataFrame(index=band_files, columns=band_files)

# Compute the correlation between each pair of bands
for i, band_file_i in tqdm(enumerate(band_files), total=len(band_files), desc='Computing correlations'):
    for j, band_file_j in enumerate(band_files[i:], start=i):  # Note: start from 'i' to include the diagonal
        pearson_corr, _ = pearsonr(band_data[band_file_i], band_data[band_file_j])
        spearman_corr, _ = spearmanr(band_data[band_file_i], band_data[band_file_j])
        correlation_df_pearson.loc[band_file_i, band_file_j] = pearson_corr
        correlation_df_pearson.loc[band_file_j, band_file_i] = pearson_corr
        correlation_df_spearman.loc[band_file_i, band_file_j] = spearman_corr
        correlation_df_spearman.loc[band_file_j, band_file_i] = spearman_corr

# Plot a heatmap of the Pearson correlation matrix
sns.heatmap(correlation_df_pearson.astype(float), annot=True, cmap='coolwarm')
plt.title('Pearson Correlation')
plt.show()

# Plot a heatmap of the Spearman correlation matrix
sns.heatmap(correlation_df_spearman.astype(float), annot=True, cmap='coolwarm')
plt.title('Spearman Correlation')
plt.show()


## plot value distribution per band

In [ ]:
band_files = ['band_08.npy', 'band_09.npy', 'band_10.npy', 'band_11.npy', 'band_12.npy', 'band_13.npy', 'band_14.npy', 'band_15.npy', 'band_16.npy']

# Loop through each band
for band_file in band_files:
    # Initialize an empty list to store values
    values = []

    # Loop through all folders in the train directory
    for folder_name in os.listdir(train_folder):
        folder_path = os.path.join(train_folder, folder_name)

        # Make sure this is a folder
        if not os.path.isdir(folder_path):
            continue

        file_path = os.path.join(folder_path, band_file)

        # Ensure the file exists
        if not os.path.exists(file_path):
            continue

        # Load the .npy file
        band_data = np.load(file_path)

        # Get the 5th image in the sequence
        fifth_image = band_data[4]  # Note: Python indexing starts from 0

        # Flatten the image to a 1D array and append it to the values list
        values.extend(fifth_image.flatten())

    # Create the histogram trace
    hist_trace = go.Histogram(
        x=values,
        nbinsx=100,
        marker=dict(
            color='blue',
            opacity=0.7
        )
    )

    # Create the layout for the subplot
    layout = go.Layout(
        title=f'Distribution of pixel values in {band_file}',
        bargap=0.1
    )

    # Create the figure with subplot
    fig = go.Figure(data=[hist_trace], layout=layout)

    # Show the figure
    fig.show()


## print a large number of bands and corresponding human masks to observbe the quality pf labeling

In [ ]:
def print_5th_band_images(input_folder, band_number1, band_number2, band_number3):
    # Convert band numbers to strings and pad with leading zeros
    band_number_str1 = str(band_number1).zfill(2)
    band_number_str2 = str(band_number2).zfill(2)
    band_number_str3 = str(band_number3).zfill(2)

    # Iterate over the subfolders in the input folder
    for subdir in os.listdir(input_folder):
        subdir_path = os.path.join(input_folder, subdir)

        # Check if the subfolder is a directory
        if os.path.isdir(subdir_path):
            print(f"Folder name: {subdir}")
            # Specify the paths to the band files
            band_file_path1 = os.path.join(subdir_path, f'band_{band_number_str1}.npy')
            band_file_path2 = os.path.join(subdir_path, f'band_{band_number_str2}.npy')
            band_file_path3 = os.path.join(subdir_path, f'band_{band_number_str3}.npy')

            # Specify the path to the pixel masks file
            masks_file_path = os.path.join(subdir_path, 'human_pixel_masks.npy')

            # Load the band files and pixel masks file
            band_data1 = np.load(band_file_path1)
            band_data2 = np.load(band_file_path2)
            band_data3 = np.load(band_file_path3)
            masks_data = np.load(masks_file_path)

            # Get the 5th image (5th dimension) from each band
            band_5th_image1 = band_data1[:, :, 4]
            band_5th_image2 = band_data2[:, :, 4]
            band_5th_image3 = band_data3[:, :, 4]

            # Get the pixel masks
            masks = masks_data[:, :, 0]

            # Create a subplot with 2 rows and 2 columns
            fig, axs = plt.subplots(2, 2, figsize=(10, 10))

            # Display the band images and mask in subplots
            sns.heatmap(band_5th_image1, cmap='Spectral', cbar=False, ax=axs[0, 0])
            axs[0, 0].set_title(f'5th Image - Band {band_number_str1}')
            axs[0, 0].axis('off')

            sns.heatmap(band_5th_image2, cmap='Spectral', cbar=False, ax=axs[0, 1])
            axs[0, 1].set_title(f'5th Image - Band {band_number_str2}')
            axs[0, 1].axis('off')

            sns.heatmap(band_5th_image3, cmap='Spectral', cbar=False, ax=axs[1, 0])
            axs[1, 0].set_title(f'5th Image - Band {band_number_str3}')
            axs[1, 0].axis('off')

            sns.heatmap(masks, cmap='Blues', cbar=False, linewidths=0.0, ax=axs[1, 1])
            axs[1, 1].set_title('Pixel Masks')
            axs[1, 1].axis('off')

            # Adjust the spacing between subplots
            plt.tight_layout()

            # Show the plot
            plt.show()

In [ ]:
print_5th_band_images(train_folder, 9, 12, 15)